# NCHASH Test Suite

In [ ]:
import numpy as np
import os
import sys

sys.path.insert(0, '/home/chuan/Code/NCHASH')

from nchash import driver, core, uncertainty, utils, io

HASH_DIR = '/home/chuan/Code/NCHASH/HASH_v1.2'

## 1. Basic Functionality Tests

In [ ]:
print("Testing basic functionality...")

# Test cross product
v1 = np.array([1.0, 0.0, 0.0])
v2 = np.array([0.0, 1.0, 0.0])
result = utils.cross_product(v1, v2)
expected = np.array([0.0, 0.0, 1.0])
assert np.allclose(result, expected), f"Cross product failed: {result} != {expected}"
print("  [OK] Cross product")

# Test spherical to Cartesian conversion
x, y, z = utils.to_cartesian(0.0, 0.0, 1.0)  # Straight up
assert abs(x) < 1e-10 and abs(y) < 1e-10 and abs(z + 1.0) < 1e-10

x, y, z = utils.to_cartesian(90.0, 0.0, 1.0)  # Horizontal, north
assert abs(x - 1.0) < 1e-10 and abs(y) < 1e-10 and abs(z) < 1e-10
print("  [OK] Spherical to Cartesian")

# Test fault plane coordinate conversion
fnorm, slip = utils.fp_coord_angles_to_vectors(0.0, 90.0, 0.0)
s, d, r = utils.fp_coord_vectors_to_angles(fnorm, slip)
assert abs(s) < 1.0 or abs(abs(s) - 360.0) < 1.0
assert abs(d - 90.0) < 1.0
print("  [OK] Fault plane coordinates")

print("\nAll basic tests passed!")

## 2. Grid Search Test

In [ ]:
print("Testing grid search algorithm...")

# Create synthetic test case: pure strike-slip
fnorm_ref, slip_ref = utils.fp_coord_angles_to_vectors(0.0, 90.0, 0.0)

npol = 20
np.random.seed(42)
p_azi = np.random.uniform(0, 360, npol)
p_the = np.random.uniform(20, 160, npol)
p_pol = np.zeros(npol, dtype=np.int32)
p_qual = np.zeros(npol, dtype=np.int32)

for i in range(npol):
    theta = p_the[i] * utils.DEG_TO_RAD
    phi = p_azi[i] * utils.DEG_TO_RAD
    p_a1 = np.sin(theta) * np.cos(phi)
    p_a2 = np.sin(theta) * np.sin(phi)
    p_a3 = -np.cos(theta)
    p_b1 = (slip_ref[0] * p_a1 + slip_ref[1] * p_a2 + slip_ref[2] * p_a3)
    p_b3 = (fnorm_ref[0] * p_a1 + fnorm_ref[1] * p_a2 + fnorm_ref[2] * p_a3)
    p_pol[i] = 1 if p_b1 * p_b3 > 0 else -1

# Add 10% noise
n_noise = int(npol * 0.1)
noise_idx = np.random.choice(npol, n_noise, replace=False)
p_pol[noise_idx] = -p_pol[noise_idx]

# Run grid search
p_azi_mc = np.tile(p_azi.reshape(-1, 1), (1, 10))
p_the_mc = np.tile(p_the.reshape(-1, 1), (1, 10))

result = core.focalmc(
    p_azi_mc, p_the_mc, p_pol, p_qual,
    npol, 10, dang=10.0, maxout=100, nextra=2,
    ntotal=max(int(npol * 0.15), 2)
)

print(f"  Found {result['nf']} acceptable mechanisms")

if result['nf'] > 0:
    prob_result = uncertainty.mech_prob(
        result['nf'], result['faults'], result['slips'],
        cangle=45.0, prob_max=0.1
    )
    if prob_result['nsltn'] > 0:
        print(f"  Input:  strike=0, dip=90, rake=0")
        print(f"  Output: strike={prob_result['strike_avg'][0]:.1f}, "
              f"dip={prob_result['dip_avg'][0]:.1f}, "
              f"rake={prob_result['rake_avg'][0]:.1f}")
        print("  [OK] Grid search found acceptable mechanism")
else:
    print("  [WARN] No acceptable mechanisms found")

## 3. Comprehensive Tests vs Fortran

In [ ]:
# Tolerance
STRIKE_TOL = 5.0
DIP_TOL = 5.0
RAKE_TOL = 10.0

def parse_expected_output(filename):
    """Parse expected HASH output file."""
    expected_events = {}
    with open(filename, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            try:
                event_id = line[0:16].strip()
                if not event_id:
                    continue
                strike = int(line[129:133]) if line[129:133].strip() else 999
                dip = int(line[134:137]) if line[134:137].strip() else 99
                rake = int(line[138:142]) if line[138:142].strip() else 999
                quality = line[159].strip()
                is_valid = strike < 900 and dip < 90
                expected_events[event_id] = {
                    'strike': strike, 'dip': dip, 'rake': rake,
                    'quality': quality, 'is_valid': is_valid
                }
            except:
                continue
    return expected_events

In [ ]:
def run_example_test(example_num):
    """Run a single example test."""
    inp_file = os.path.join(HASH_DIR, f'example{example_num}.inp')
    expected_out_file = os.path.join(HASH_DIR, f'example{example_num}.out')
    
    if not os.path.exists(inp_file) or not os.path.exists(expected_out_file):
        return None
    
    params = io.read_hash_input_file(inp_file)
    
    # Handle example 5 with multiple phase files
    events = []
    if example_num == 5:
        with open(inp_file, 'r') as f:
            lines = f.readlines()
        if len(lines) > 2:
            for pf in [lines[1].strip(), lines[2].strip()]:
                if pf and os.path.exists(os.path.join(HASH_DIR, pf)):
                    for ev in io.read_phase_file(os.path.join(HASH_DIR, pf)):
                        if ev['id'].strip() not in [e['id'].strip() for e in events]:
                            events.append(ev)
    else:
        events = io.read_phase_file(os.path.join(HASH_DIR, params['phasefile']))
    
    stations = io.read_station_file(os.path.join(HASH_DIR, 'scsn.stations'))
    pol_file = os.path.join(HASH_DIR, params['polfile'])
    reversals = io.read_polarity_reversal_file(pol_file) if os.path.exists(pol_file) else {}
    
    expected = parse_expected_output(expected_out_file)
    
    matches, close, mismatches = 0, 0, 0
    
    for event in events:
        event_id = event['id'].strip()
        if event_id not in expected:
            continue
        
        exp = expected[event_id]
        result = driver.process_event(event, stations, reversals, params)
        
        if not exp['is_valid']:
            if not result.get('success'):
                matches += 1
            else:
                mismatches += 1
        elif result.get('success'):
            strike_ok = abs(result.get('strike_avg', 0) - exp['strike']) <= STRIKE_TOL
            dip_ok = abs(result.get('dip_avg', 0) - exp['dip']) <= DIP_TOL
            rake_ok = abs(result.get('rake_avg', 0) - exp['rake']) <= RAKE_TOL
            
            if strike_ok and dip_ok and rake_ok:
                matches += 1
            elif strike_ok and dip_ok:
                close += 1
            else:
                mismatches += 1
        else:
            mismatches += 1
    
    total = matches + close + mismatches
    return {'matches': matches, 'close': close, 'mismatches': mismatches, 'total': total}

In [ ]:
print("="*60)
print("Running comprehensive tests (Python vs Fortran)")
print("="*60)

all_results = {}

for i in range(1, 6):
    result = run_example_test(i)
    if result:
        all_results[f'example{i}'] = result
        rate = (result['matches'] + result['close']) / result['total'] * 100 if result['total'] > 0 else 0
        print(f"\nExample {i}: {result['matches']}/{result['total']} matches ({rate:.0f}%)")
        print(f"  Matches: {result['matches']}, Close: {result['close']}, Mismatches: {result['mismatches']}")
    else:
        print(f"\nExample {i}: Skipped (files not found)")

# Summary
print("\n" + "="*60)
print("OVERALL SUMMARY")
print("="*60)

total_matches = sum(r['matches'] for r in all_results.values())
total_close = sum(r['close'] for r in all_results.values())
total_events = sum(r['total'] for r in all_results.values())

if total_events > 0:
    overall_rate = (total_matches + total_close) / total_events * 100
    print(f"\nTotal: {total_matches + total_close}/{total_events} ({overall_rate:.0f}%)")